# Dataset
Source: https://www.kaggle.com/c/plant-pathology-2020-fgvc7

In [ ]:
import os

# Dataset path
mainDir = "plant-pathology-2020-fgvc7"
imagesDir = os.path.join(mainDir, "images")
trainFile = os.path.join(mainDir, "train.csv")

# Get list of all images
imagesFileList = [os.path.join(imagesDir, f) for f in os.listdir(imagesDir) 
                  if os.path.isfile(os.path.join(imagesDir, f))]

In [ ]:
import pandas as pd
from PIL import Image

# Basic information of dataset
imageSizeStat = {}
for imageFile in imagesFileList:
    with Image.open(imageFile) as im:
        imageSize = im.size
        if imageSize not in imageSizeStat:
            imageSizeStat[imageSize] = 1
        else:
            imageSizeStat[imageSize] += 1
print("Size:")
print("\t{}".format(imageSizeStat))

trainDF = pd.read_csv(trainFile)
print("Label:")
print("\thealthy: {}".format(trainDF["healthy"].sum()))
print("\tmultiple_diseases: {}".format(trainDF["multiple_diseases"].sum()))
print("\trust: {}".format(trainDF["rust"].sum()))
print("\tscab: {}".format(trainDF["scab"].sum()))
print("\t==> Total: {}".format(trainDF.shape[0]))

##### Size:
(2048, 1365): 3620\
(1365, 2048): 22\
==> Can keep size but need to rotate to same direction

We have:
\begin{equation}
\frac{2048}{1365} \approx \frac{3}{2}
\end{equation}
==> Size ratio: 3:2

To avoid running out of memory, crop images with size 1920:1280, then resize images to 384:256.
##### Label:
healthy: 516\
multiple_diseases: 91\
rust: 622\
scab: 592\
==> Not balanced\
==> Need to rebalance

**Note: healthy means no rust and no scab, while multiple_diseases means both rust and scab.**\
==> Build 2 models: one for rust and one for scab\
==> healthy(0,0); multiple_diseases(1,1); rust(1,0); scab(0,1)

Using data augmentation to create training and validation set as follows:\
&emsp;Training set: rust(0,60,300,0 : 300,0,0,60); scab(0,60,0,300 : 300,0,60,0)\
&emsp;Validation set: rust(0,20,100,0 : 100,0,0,20); scab(0,20,0,100 : 100,0,20,0)

# Baseline

In [ ]:
import random
import numpy as np
from sklearn.model_selection import train_test_split

def split_dataset(image_id, total):
    train, test = train_test_split(image_id, test_size=0.25)
    train = train.sample(n=total*3//4)
    test = test.sample(n=total//4)
    return train, test

def data_augmentation(im):
    # Rotate to same direction
    if im.size[0] < im.size[1]:
        im = im.transpose(Image.ROTATE_90)
    # Random flip
    if random.getrandbits(1):
        im = im.transpose(Image.FLIP_LEFT_RIGHT)
    if random.getrandbits(1):
        im = im.transpose(Image.FLIP_TOP_BOTTOM)
    # Random crop with 1920:1280 size
    left = random.randint(0,2048-1920)
    top = random.randint(0,1365-1280)
    right = left + 1920
    bottom = top + 1280
    im = im.crop((left, top, right, bottom))
    # Resize to 384:256 size
    im = im.resize((384,256))
    return im

def create_dataset(FF, TT, TF, FT):
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []

    FF_train, FF_test = split_dataset(FF, 400)
    TT_train, TT_test = split_dataset(TT, 80)
    TF_train, TF_test = split_dataset(TF, 400)
    FT_train, FT_test = split_dataset(FT, 80)

    for image_id in FF_train:
        path = os.path.join(imagesDir, image_id + ".jpg")
        im = Image.open(path)
        im = data_augmentation(im)
        im = np.asarray(im)
        train_images.append(im)
        train_labels.append([np.uint8(0)])
    for image_id in FF_test:
        path = os.path.join(imagesDir, image_id + ".jpg")
        im = Image.open(path)
        im = data_augmentation(im)
        im = np.asarray(im)
        test_images.append(im)
        test_labels.append([np.uint8(0)])
    for image_id in TT_train:
        path = os.path.join(imagesDir, image_id + ".jpg")
        im = Image.open(path)
        im = data_augmentation(im)
        im = np.asarray(im)
        train_images.append(im)
        train_labels.append([np.uint8(1)])
    for image_id in TT_test:
        path = os.path.join(imagesDir, image_id + ".jpg")
        im = Image.open(path)
        im = data_augmentation(im)
        im = np.asarray(im)
        test_images.append(im)
        test_labels.append([np.uint8(1)])
    for image_id in TF_train:
        path = os.path.join(imagesDir, image_id + ".jpg")
        im = Image.open(path)
        im = data_augmentation(im)
        im = np.asarray(im)
        train_images.append(im)
        train_labels.append([np.uint8(1)])
    for image_id in TF_test:
        path = os.path.join(imagesDir, image_id + ".jpg")
        im = Image.open(path)
        im = data_augmentation(im)
        im = np.asarray(im)
        test_images.append(im)
        test_labels.append([np.uint8(1)])
    for image_id in FT_train:
        path = os.path.join(imagesDir, image_id + ".jpg")
        im = Image.open(path)
        im = data_augmentation(im)
        im = np.asarray(im)
        train_images.append(im)
        train_labels.append([np.uint8(0)])
    for image_id in FT_test:
        path = os.path.join(imagesDir, image_id + ".jpg")
        im = Image.open(path)
        im = data_augmentation(im)
        im = np.asarray(im)
        test_images.append(im)
        test_labels.append([np.uint8(0)])

    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)
    return (train_images, train_labels), (test_images, test_labels)

# Split dataset
trainDF = pd.read_csv(trainFile)
healthy = trainDF[trainDF["healthy"] == 1]["image_id"]
multiple_diseases = trainDF[trainDF["multiple_diseases"] == 1]["image_id"]
rust = trainDF[trainDF["rust"] == 1]["image_id"]
scab = trainDF[trainDF["scab"] == 1]["image_id"]

# Pre-process dataset
(rust_train_images, rust_train_labels), (rust_test_images, rust_test_labels) = create_dataset(
    healthy, multiple_diseases, rust, scab
)
(scab_train_images, scab_train_labels), (scab_test_images, scab_test_labels) = create_dataset(
    healthy, multiple_diseases, scab, rust
)
np.save(os.path.join(mainDir, "rust_train_images.npy"), rust_train_images)
np.save(os.path.join(mainDir, "rust_train_labels.npy"), rust_train_labels)
np.save(os.path.join(mainDir, "rust_test_images.npy"), rust_test_images)
np.save(os.path.join(mainDir, "rust_test_labels.npy"), rust_test_labels)
np.save(os.path.join(mainDir, "scab_train_images.npy"), scab_train_images)
np.save(os.path.join(mainDir, "scab_train_labels.npy"), scab_train_labels)
np.save(os.path.join(mainDir, "scab_test_images.npy"), scab_test_images)
np.save(os.path.join(mainDir, "scab_test_labels.npy"), scab_test_labels)

In [ ]:
import numpy as np

def get_information(images, labels):
    print(images.shape, labels.shape)

# Load pre-processed dataset
rust_train_images = np.load(os.path.join(mainDir, "rust_train_images.npy"))
rust_train_labels = np.load(os.path.join(mainDir, "rust_train_labels.npy"))
rust_test_images = np.load(os.path.join(mainDir, "rust_test_images.npy"))
rust_test_labels = np.load(os.path.join(mainDir, "rust_test_labels.npy"))
scab_train_images = np.load(os.path.join(mainDir, "scab_train_images.npy"))
scab_train_labels = np.load(os.path.join(mainDir, "scab_train_labels.npy"))
scab_test_images = np.load(os.path.join(mainDir, "scab_test_images.npy"))
scab_test_labels = np.load(os.path.join(mainDir, "scab_test_labels.npy"))

# Get information
get_information(rust_train_images, rust_train_labels)
get_information(rust_test_images, rust_test_labels)
get_information(scab_train_images, scab_train_labels)
get_information(scab_test_images, scab_test_labels)

# Normalize pixel values to be between 0 and 1
rust_train_images = rust_train_images / 255.0
rust_test_images = rust_test_images / 255.0
scab_train_images = scab_train_images / 255.0
scab_test_images = scab_test_images / 255.0

In [ ]:
from tensorflow.keras import layers, models

def block(x, filters, strides, conv_shortcut):
    if conv_shortcut:
        shortcut = layers.Conv2D(filters, 1, strides=strides, padding='same')(x)
        shortcut = layers.BatchNormalization()(shortcut)
    else:
        shortcut = x
    
    x = layers.Conv2D(filters, 3, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)
    
    return x

def stack(x, filters, blocks, strides1):
    x = block(x, filters, strides=strides1, conv_shortcut=True)
    for i in range(1, blocks):
        x = block(x, filters, strides=1, conv_shortcut=False)
    return x

def create_model(dropout_rate):
    inputs = layers.Input(shape=(256, 384, 3))

    x = layers.Conv2D(32, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(2, strides=2, padding='same')(x)

    x = stack(x, 32, 3, 1)
    x = layers.Dropout(dropout_rate)(x)
    x = stack(x, 64, 4, 2)
    x = layers.Dropout(dropout_rate)(x)
    x = stack(x, 128, 6, 2)
    x = layers.Dropout(dropout_rate)(x)
    x = stack(x, 256, 3, 2)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    return models.Model(inputs=inputs, outputs=outputs)

##### Rust Model

In [ ]:
rust = create_model(0.1)
rust.summary()
rust.compile(optimizer='SGD', loss="binary_crossentropy", metrics="binary_accuracy")

In [ ]:
from tensorflow.keras import callbacks

rust_history = rust.fit(rust_train_images, rust_train_labels, epochs=100,
                        validation_data=(rust_test_images, rust_test_labels),
                        callbacks=[callbacks.ModelCheckpoint(os.path.join(mainDir, "rust_model"),
                                                             save_best_only=True)])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(rust_history.history['binary_accuracy'], label='binary_accuracy')
plt.plot(rust_history.history['val_binary_accuracy'], label = 'val_binary_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
import numpy as np
from tensorflow.keras import models

rust = models.load_model(os.path.join(mainDir, "rust_model"))
rust_test_loss, rust_test_acc = rust.evaluate(rust_test_images,  rust_test_labels, verbose=2)
rust_pred = rust.predict(rust_test_images).reshape(-1)
print(np.histogram(rust_pred, range=(0,1))[0])

##### Scab Model

In [ ]:
scab = create_model(0.3)
scab.summary()
scab.compile(optimizer='SGD', loss="binary_crossentropy", metrics="binary_accuracy")

In [ ]:
from tensorflow.keras import callbacks

scab_history = scab.fit(scab_train_images, scab_train_labels, epochs=100,
                        validation_data=(scab_test_images, scab_test_labels),
                        callbacks=[callbacks.ModelCheckpoint(os.path.join(mainDir, "scab_model"),
                                                             save_best_only=True)])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(scab_history.history['binary_accuracy'], label='binary_accuracy')
plt.plot(scab_history.history['val_binary_accuracy'], label = 'val_binary_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
import numpy as np
from tensorflow.keras import models

scab = models.load_model(os.path.join(mainDir, "scab_model"))
scab_test_loss, scab_test_acc = scab.evaluate(scab_test_images,  scab_test_labels, verbose=2)
scab_pred = scab.predict(scab_test_images).reshape(-1)
print(np.histogram(scab_pred, range=(0,1))[0])

##### Predict Test Set

In [ ]:
import os
import pandas as pd

# Test set path
mainDir = "plant-pathology-2020-fgvc7"
imagesDir = os.path.join(mainDir, "images")
testFile = os.path.join(mainDir, "sample_submission.csv")

# Test set DF
testDF = pd.read_csv(testFile)

In [ ]:
from tensorflow.keras import models

rust = models.load_model(os.path.join(mainDir, "rust_model"))
scab = models.load_model(os.path.join(mainDir, "scab_model"))

In [ ]:
import random
import numpy as np

def data_augmentation(im):
    # Rotate to same direction
    if im.size[0] < im.size[1]:
        im = im.transpose(Image.ROTATE_90)
    # Random flip
    if random.getrandbits(1):
        im = im.transpose(Image.FLIP_LEFT_RIGHT)
    if random.getrandbits(1):
        im = im.transpose(Image.FLIP_TOP_BOTTOM)
    # Random crop with 1920:1280 size
    left = random.randint(0,2048-1920)
    top = random.randint(0,1365-1280)
    right = left + 1920
    bottom = top + 1280
    im = im.crop((left, top, right, bottom))
    # Resize to 384:256 size
    im = im.resize((384,256))
    return im

test_images = []
for image_id in testDF["image_id"]:
    path = os.path.join(imagesDir, image_id + ".jpg")
    im = Image.open(path)
    im = data_augmentation(im)
    im = np.asarray(im)
    test_images.append(im)
test_images = np.array(test_images)
np.save(os.path.join(mainDir, "test_images.npy"), test_images)

In [ ]:
import numpy as np

# Load pre-processed dataset
test_images = np.load(os.path.join(mainDir, "test_images.npy"))

# Get information
print(test_images.shape)

# Normalize pixel values to be between 0 and 1
test_images = test_images / 255.0

In [ ]:
rust_pred = rust.predict(test_images).reshape(-1)
scab_pred = scab.predict(test_images).reshape(-1)

In [ ]:
testDF["healthy"] = (1 - rust_pred) * (1 - scab_pred)
testDF["multiple_diseases"] = rust_pred * scab_pred
testDF["rust"] = rust_pred * (1 - scab_pred)
testDF["scab"] = (1 - rust_pred) * scab_pred

testDF.to_csv(os.path.join(mainDir, "submission.csv"), float_format="%.4f", header=True, index=False)